# Part 2: Markov Decision Processes and Bellman Equations

In this notebook, we'll formalize the reinforcement learning problem using **Markov Decision Processes (MDPs)** and derive the fundamental **Bellman equations**.

## What You'll Learn
- Markov Processes (Markov Chains)
- Markov Reward Processes (MRP)
- Markov Decision Processes (MDP)
- The Return and Value Functions
- Bellman Expectation Equations
- Optimal Value Functions and Bellman Optimality Equations

Let's begin!

## Setup

In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

# Set style for plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)

print("Setup complete!")

---
# 1. Markov Processes (Markov Chains)

Before we get to MDPs, let's start with simpler structures.

## Definition

A **Markov Process** (or Markov Chain) is a tuple $(S, P)$ where:
- $S$ is a finite set of states
- $P$ is a state transition probability matrix: $P_{ss'} = P[S_{t+1} = s' | S_t = s]$

The Markov Process describes a random walk through states, where the next state depends only on the current state (Markov property).

## State Transition Matrix

The transition matrix $P$ defines the probability of moving from any state $s$ to any state $s'$:

$$P = \begin{pmatrix} P_{11} & P_{12} & \cdots & P_{1n} \\ P_{21} & P_{22} & \cdots & P_{2n} \\ \vdots & \vdots & \ddots & \vdots \\ P_{n1} & P_{n2} & \cdots & P_{nn} \end{pmatrix}$$

Each row sums to 1 (we must go somewhere).

In [ ]:
# Example: Simple Weather Markov Chain
# States: 0 = Sunny, 1 = Rainy

states = ['Sunny', 'Rainy']

# Transition matrix
# P[i,j] = probability of going from state i to state j
P_weather = np.array([
    [0.8, 0.2],  # From Sunny: 80% stay sunny, 20% become rainy
    [0.4, 0.6]   # From Rainy: 40% become sunny, 60% stay rainy
])

print("Weather Markov Chain")
print("=" * 40)
print("\nTransition Matrix P:")
print(P_weather)
print(f"\nRow sums (should be 1.0): {P_weather.sum(axis=1)}")

# Visualize
fig, ax = plt.subplots(figsize=(6, 5))
sns.heatmap(P_weather, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=states, yticklabels=states, ax=ax)
ax.set_title('Weather Transition Matrix')
ax.set_xlabel('Next State')
ax.set_ylabel('Current State')
plt.tight_layout()
plt.show()

In [ ]:
# Simulate a Markov Chain
def simulate_markov_chain(P, initial_state, n_steps):
    """Simulate a Markov chain for n_steps."""
    states_sequence = [initial_state]
    current_state = initial_state
    
    for _ in range(n_steps):
        # Sample next state according to transition probabilities
        next_state = np.random.choice(len(P), p=P[current_state])
        states_sequence.append(next_state)
        current_state = next_state
    
    return states_sequence

# Simulate 30 days of weather
np.random.seed(42)
weather_sequence = simulate_markov_chain(P_weather, initial_state=0, n_steps=30)

print("Simulated Weather for 30 Days")
print("=" * 50)
weather_names = [states[s] for s in weather_sequence]
print(f"Day 1-10:  {weather_names[:10]}")
print(f"Day 11-20: {weather_names[10:20]}")
print(f"Day 21-31: {weather_names[20:]}")

# Count occurrences
sunny_count = weather_sequence.count(0)
rainy_count = weather_sequence.count(1)
print(f"\nSunny days: {sunny_count} ({sunny_count/len(weather_sequence)*100:.1f}%)")
print(f"Rainy days: {rainy_count} ({rainy_count/len(weather_sequence)*100:.1f}%)")

In [ ]:
# Visualize the weather sequence
fig, ax = plt.subplots(figsize=(14, 3))

colors = ['gold' if s == 0 else 'steelblue' for s in weather_sequence]
ax.bar(range(len(weather_sequence)), [1]*len(weather_sequence), color=colors, edgecolor='black')
ax.set_xticks(range(0, len(weather_sequence), 5))
ax.set_xlabel('Day')
ax.set_yticks([])
ax.set_title('Weather Simulation: 31 Days')

# Legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='gold', label='Sunny'),
                   Patch(facecolor='steelblue', label='Rainy')]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

---
# 2. Markov Reward Process (MRP)

A **Markov Reward Process** is a Markov Chain with values (rewards).

## Definition

An MRP is a tuple $(S, P, R, \gamma)$ where:
- $S$ is a finite set of states
- $P$ is the state transition probability matrix
- $R$ is a reward function: $R_s = E[R_{t+1} | S_t = s]$ (expected reward in state $s$)
- $\gamma$ is a discount factor, $\gamma \in [0, 1]$

The difference from a Markov Process: we now get rewards!

In [ ]:
# Example: Student MRP (from David Silver's course)
# States represent a student's activities

mrp_states = ['Facebook', 'Class 1', 'Class 2', 'Class 3', 'Pass', 'Pub', 'Sleep']
n_states = len(mrp_states)

# Transition matrix
P_student = np.array([
    # FB    C1    C2    C3    Pass  Pub   Sleep
    [0.9,  0.1,  0.0,  0.0,  0.0,  0.0,  0.0],   # Facebook
    [0.5,  0.0,  0.5,  0.0,  0.0,  0.0,  0.0],   # Class 1
    [0.0,  0.0,  0.0,  0.8,  0.0,  0.0,  0.2],   # Class 2
    [0.0,  0.0,  0.0,  0.0,  0.6,  0.4,  0.0],   # Class 3
    [0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  1.0],   # Pass (terminal)
    [0.2,  0.4,  0.4,  0.0,  0.0,  0.0,  0.0],   # Pub
    [0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  1.0],   # Sleep (terminal)
])

# Rewards for being in each state
R_student = np.array([-1, -2, -2, -2, 10, 1, 0])

print("Student MRP")
print("=" * 50)
print("\nStates and Rewards:")
for i, (state, reward) in enumerate(zip(mrp_states, R_student)):
    print(f"  State {i} ({state}): Reward = {reward}")

print("\nTransition probabilities from Class 3:")
for j, prob in enumerate(P_student[3]):
    if prob > 0:
        print(f"  → {mrp_states[j]}: {prob:.1f}")

## The Return $G_t$

The **return** $G_t$ is the total discounted reward from time-step $t$:

$$G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \ldots = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}$$

The discount factor $\gamma$ determines the present value of future rewards:
- $\gamma = 0$: Only immediate reward matters
- $\gamma = 1$: Future rewards are as important as immediate (undiscounted)
- Typically $\gamma \in [0.9, 0.99]$

In [ ]:
# Simulate the student MRP and calculate returns
def simulate_mrp(P, R, initial_state, max_steps=100):
    """Simulate an MRP episode, returning states and rewards."""
    states = [initial_state]
    rewards = []
    current_state = initial_state
    
    for _ in range(max_steps):
        # Get reward for current state
        reward = R[current_state]
        rewards.append(reward)
        
        # Check for terminal states (Sleep or Pass - self-loops with prob 1)
        if current_state in [4, 6]:  # Pass or Sleep
            break
        
        # Transition to next state
        next_state = np.random.choice(len(P), p=P[current_state])
        states.append(next_state)
        current_state = next_state
    
    return states, rewards

def compute_return(rewards, gamma):
    """Compute the discounted return from a sequence of rewards."""
    G = 0
    for t, r in enumerate(rewards):
        G += (gamma ** t) * r
    return G

# Simulate several episodes starting from Class 1
np.random.seed(42)
gamma = 0.9

print("Student MRP: Sample Episodes from Class 1")
print("=" * 60)
print(f"Discount factor γ = {gamma}\n")

for episode in range(5):
    states, rewards = simulate_mrp(P_student, R_student, initial_state=1)
    G = compute_return(rewards, gamma)
    
    path = ' → '.join([mrp_states[s] for s in states])
    print(f"Episode {episode + 1}:")
    print(f"  Path: {path}")
    print(f"  Rewards: {rewards}")
    print(f"  Return G = {G:.2f}")
    print()

## Value Function for MRP

The **state value function** $v(s)$ gives the expected return starting from state $s$:

$$v(s) = E[G_t | S_t = s]$$

This tells us how "good" it is to be in a particular state.

In [ ]:
# Estimate value function using Monte Carlo simulation
def estimate_value_mc(P, R, gamma, n_episodes=10000):
    """Estimate value function using Monte Carlo sampling."""
    n_states = len(R)
    returns_sum = np.zeros(n_states)
    returns_count = np.zeros(n_states)
    
    for _ in range(n_episodes):
        # Start from a random non-terminal state
        initial_state = np.random.choice([0, 1, 2, 3, 5])  # Exclude terminal states
        states, rewards = simulate_mrp(P, R, initial_state)
        
        # Calculate returns for each visited state
        G = 0
        for t in reversed(range(len(rewards))):
            G = rewards[t] + gamma * G
            state = states[t]
            returns_sum[state] += G
            returns_count[state] += 1
    
    # Average returns
    V = np.zeros(n_states)
    for s in range(n_states):
        if returns_count[s] > 0:
            V[s] = returns_sum[s] / returns_count[s]
    
    return V

# Estimate values
V_mc = estimate_value_mc(P_student, R_student, gamma=0.9, n_episodes=50000)

print("Estimated State Values (Monte Carlo, γ=0.9)")
print("=" * 50)
for i, (state, value) in enumerate(zip(mrp_states, V_mc)):
    print(f"  V({state}) = {value:.2f}")

In [ ]:
# Visualize state values
fig, ax = plt.subplots(figsize=(10, 5))

colors = ['red' if v < 0 else 'green' for v in V_mc]
bars = ax.bar(mrp_states, V_mc, color=colors, edgecolor='black')

ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.set_ylabel('Value V(s)')
ax.set_title('State Values in Student MRP (γ = 0.9)')

# Add value labels
for bar, val in zip(bars, V_mc):
    ypos = bar.get_height() + 0.3 if val >= 0 else bar.get_height() - 0.5
    ax.text(bar.get_x() + bar.get_width()/2, ypos, f'{val:.1f}',
            ha='center', va='bottom' if val >= 0 else 'top', fontsize=10)

plt.tight_layout()
plt.show()

## Bellman Equation for MRP

The value function can be decomposed into two parts:
1. Immediate reward $R_{t+1}$
2. Discounted value of the successor state $\gamma v(S_{t+1})$

$$v(s) = E[R_{t+1} + \gamma v(S_{t+1}) | S_t = s]$$

Expanding:

$$v(s) = R_s + \gamma \sum_{s' \in S} P_{ss'} v(s')$$

### Matrix Form

The Bellman equation can be written in matrix form:

$$v = R + \gamma P v$$

This is a linear equation that can be solved directly:

$$(I - \gamma P) v = R$$
$$v = (I - \gamma P)^{-1} R$$

This direct solution has complexity $O(n^3)$ and is only practical for small state spaces.

In [ ]:
# Solve Bellman equation directly using matrix inversion
def solve_mrp_bellman(P, R, gamma):
    """Solve MRP Bellman equation: v = (I - gamma*P)^(-1) * R"""
    n = len(R)
    I = np.eye(n)
    V = np.linalg.solve(I - gamma * P, R)
    return V

# Solve directly
V_direct = solve_mrp_bellman(P_student, R_student, gamma=0.9)

print("State Values: Direct Solution vs Monte Carlo")
print("=" * 60)
print(f"{'State':<12} {'Direct':>12} {'Monte Carlo':>12} {'Difference':>12}")
print("-" * 60)
for i, state in enumerate(mrp_states):
    diff = abs(V_direct[i] - V_mc[i])
    print(f"{state:<12} {V_direct[i]:>12.2f} {V_mc[i]:>12.2f} {diff:>12.2f}")

In [ ]:
# Verify Bellman equation for a specific state
state = 1  # Class 1
gamma = 0.9

print(f"Verifying Bellman Equation for State '{mrp_states[state]}'")
print("=" * 60)
print(f"\nBellman equation: v(s) = R_s + γ * Σ P(s'|s) * v(s')")
print(f"\nFor {mrp_states[state]}:")
print(f"  R_s = {R_student[state]}")
print(f"  γ = {gamma}")

# Calculate expected value of successor states
expected_next_value = 0
print(f"\n  Expected next state value:")
for next_state in range(n_states):
    prob = P_student[state, next_state]
    if prob > 0:
        contribution = prob * V_direct[next_state]
        expected_next_value += contribution
        print(f"    P({mrp_states[next_state]}|{mrp_states[state]}) × V({mrp_states[next_state]}) = {prob:.1f} × {V_direct[next_state]:.2f} = {contribution:.2f}")

print(f"\n  Sum: {expected_next_value:.2f}")

# Calculate total
calculated_value = R_student[state] + gamma * expected_next_value
print(f"\n  v({mrp_states[state]}) = {R_student[state]} + {gamma} × {expected_next_value:.2f} = {calculated_value:.2f}")
print(f"  Direct solution: V({mrp_states[state]}) = {V_direct[state]:.2f}")
print(f"\n  ✓ Bellman equation verified!" if abs(calculated_value - V_direct[state]) < 0.01 else "  ✗ Error!")

---
# 3. Markov Decision Process (MDP)

A **Markov Decision Process** is an MRP with decisions (actions). It's the formal framework for reinforcement learning.

## Definition

An MDP is a tuple $(S, A, P, R, \gamma)$ where:
- $S$ is a finite set of states
- $A$ is a finite set of actions
- $P$ is the state transition probability matrix: $P_{ss'}^a = P[S_{t+1} = s' | S_t = s, A_t = a]$
- $R$ is a reward function: $R_s^a = E[R_{t+1} | S_t = s, A_t = a]$
- $\gamma$ is a discount factor, $\gamma \in [0, 1]$

**Key difference from MRP**: The agent can choose actions, and both transitions and rewards depend on the action taken.

In [ ]:
# FrozenLake as an MDP
env = gym.make("FrozenLake-v1", is_slippery=True)
env.reset()

print("FrozenLake MDP Components")
print("=" * 50)
print(f"\nS (State Space): {env.observation_space}")
print(f"   Number of states: {env.observation_space.n}")
print(f"\nA (Action Space): {env.action_space}")
print(f"   Number of actions: {env.action_space.n}")
print(f"   Actions: 0=LEFT, 1=DOWN, 2=RIGHT, 3=UP")
print(f"\nγ (Discount Factor): Typically 0.99 for this environment")

# Show the map
print("\nEnvironment Map:")
desc = env.unwrapped.desc.astype(str)
for row in desc:
    print("   " + " ".join(row))

In [ ]:
# Extract and examine the MDP's P and R
def extract_mdp_components(env):
    """Extract P (transitions) and R (rewards) from a Gymnasium environment."""
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    
    # P[s][a] = list of (prob, next_state, reward, done)
    P = env.unwrapped.P
    
    # Create transition and reward matrices
    # T[s, a, s'] = probability of s -> s' given action a
    T = np.zeros((n_states, n_actions, n_states))
    R = np.zeros((n_states, n_actions))
    
    for s in range(n_states):
        for a in range(n_actions):
            for prob, next_s, reward, done in P[s][a]:
                T[s, a, next_s] += prob
                R[s, a] += prob * reward  # Expected reward
    
    return T, R

T, R = extract_mdp_components(env)

print("MDP Transition Matrix T[s,a,s']")
print("=" * 50)
print(f"Shape: {T.shape} (states × actions × next_states)")

# Show transitions from state 6
state = 6
action_names = ['LEFT', 'DOWN', 'RIGHT', 'UP']
print(f"\nTransitions from state {state}:")
for a in range(4):
    print(f"\n  Action {a} ({action_names[a]}):")
    for next_s in range(16):
        if T[state, a, next_s] > 0:
            print(f"    → State {next_s}: P = {T[state, a, next_s]:.2f}")

In [ ]:
# Visualize transition probabilities from one state
def visualize_mdp_transitions(env, state):
    """Visualize transitions from a state for all actions."""
    desc = env.unwrapped.desc.astype(str)
    nrow, ncol = desc.shape
    action_names = ['LEFT', 'DOWN', 'RIGHT', 'UP']
    
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    colors = {'S': 'lightblue', 'F': 'white', 'H': 'lightcoral', 'G': 'lightgreen'}
    
    for action, ax in enumerate(axes):
        transitions = env.unwrapped.P[state][action]
        trans_dict = {ns: p for p, ns, _, _ in transitions if p > 0}
        
        for i in range(nrow):
            for j in range(ncol):
                cell = desc[i, j]
                state_idx = i * ncol + j
                
                if state_idx == state:
                    facecolor = 'yellow'
                elif state_idx in trans_dict:
                    prob = trans_dict[state_idx]
                    facecolor = plt.cm.Blues(0.3 + 0.7 * prob)
                else:
                    facecolor = colors.get(cell, 'white')
                
                rect = plt.Rectangle((j, nrow-1-i), 1, 1, fill=True,
                                     facecolor=facecolor, edgecolor='black')
                ax.add_patch(rect)
                
                # Add text
                ax.text(j + 0.5, nrow - 1 - i + 0.7, cell,
                       ha='center', va='center', fontsize=12)
                
                if state_idx in trans_dict:
                    ax.text(j + 0.5, nrow - 1 - i + 0.3, f'{trans_dict[state_idx]:.2f}',
                           ha='center', va='center', fontsize=10, color='blue', fontweight='bold')
        
        ax.set_xlim(0, ncol)
        ax.set_ylim(0, nrow)
        ax.set_aspect('equal')
        ax.axis('off')
        ax.set_title(f'Action: {action_names[action]}')
    
    plt.suptitle(f'Transition Probabilities from State {state} (Yellow)\n(Blue intensity = probability)', 
                fontsize=12, y=1.05)
    plt.tight_layout()
    return fig

visualize_mdp_transitions(env, state=6)
plt.show()

print("Notice: Due to slippery ice, the intended action only succeeds 1/3 of the time!")
print("The agent might slip and move perpendicular to the intended direction.")

---
# 4. Policies in MDPs

A **policy** $\pi$ is a distribution over actions given states:

$$\pi(a|s) = P[A_t = a | S_t = s]$$

Properties:
- A policy fully defines an agent's behavior
- MDP policies depend only on the current state (not history) - they are **stationary**
- Given an MDP and a policy, we get an MRP

## From MDP + Policy to MRP

Given MDP $M = (S, A, P, R, \gamma)$ and policy $\pi$:

The state sequence $S_1, S_2, \ldots$ is a Markov process with:

$$P_{ss'}^\pi = \sum_{a \in A} \pi(a|s) P_{ss'}^a$$

$$R_s^\pi = \sum_{a \in A} \pi(a|s) R_s^a$$

In [ ]:
# Define different policies for FrozenLake
n_states = env.observation_space.n
n_actions = env.action_space.n

# Uniform random policy: equal probability for all actions
def uniform_policy(n_actions):
    """Return a uniform random policy."""
    policy = np.ones((n_states, n_actions)) / n_actions
    return policy

# Deterministic policy: always go right then down
def right_down_policy():
    """Policy that prefers RIGHT, then DOWN."""
    # Encoded as probabilities for each action [LEFT, DOWN, RIGHT, UP]
    policy = np.zeros((n_states, n_actions))
    for s in range(n_states):
        row, col = s // 4, s % 4
        if col < 3:  # Can go right
            policy[s, 2] = 1.0  # RIGHT
        else:  # At rightmost column, go down
            policy[s, 1] = 1.0  # DOWN
    return policy

pi_random = uniform_policy(n_actions)
pi_right_down = right_down_policy()

print("Example Policies")
print("=" * 50)
print("\nUniform Random Policy π_random(a|s=0):")
for a in range(n_actions):
    print(f"  π({action_names[a]}|s=0) = {pi_random[0, a]:.2f}")

print("\nRight-then-Down Policy π_rd(a|s=0):")
for a in range(n_actions):
    print(f"  π({action_names[a]}|s=0) = {pi_right_down[0, a]:.2f}")

In [ ]:
# Convert MDP + Policy to MRP
def mdp_to_mrp(T, R_sa, policy):
    """Convert MDP with policy to MRP.
    
    Args:
        T: Transition matrix T[s,a,s']
        R_sa: Reward matrix R[s,a]
        policy: Policy matrix π[s,a]
    
    Returns:
        P_mrp: MRP transition matrix P[s,s']
        R_mrp: MRP reward vector R[s]
    """
    n_states = T.shape[0]
    n_actions = T.shape[1]
    
    # P_ss'^π = Σ_a π(a|s) * P_ss'^a
    P_mrp = np.zeros((n_states, n_states))
    for s in range(n_states):
        for a in range(n_actions):
            P_mrp[s] += policy[s, a] * T[s, a]
    
    # R_s^π = Σ_a π(a|s) * R_s^a
    R_mrp = np.sum(policy * R_sa, axis=1)
    
    return P_mrp, R_mrp

# Convert FrozenLake MDP with random policy to MRP
P_mrp, R_mrp = mdp_to_mrp(T, R, pi_random)

print("MDP + Random Policy → MRP")
print("=" * 50)
print(f"\nMRP Transition Matrix shape: {P_mrp.shape}")
print(f"MRP Reward Vector shape: {R_mrp.shape}")
print(f"\nRow sums of P_mrp (should be 1): {P_mrp.sum(axis=1)[:4]}...")

---
# 5. Value Functions for MDPs

## State-Value Function $V^\pi(s)$

The **state-value function** $V^\pi(s)$ is the expected return starting from state $s$ and following policy $\pi$:

$$V^\pi(s) = E_\pi[G_t | S_t = s]$$

## Action-Value Function $Q^\pi(s, a)$

The **action-value function** $Q^\pi(s, a)$ is the expected return starting from state $s$, taking action $a$, then following policy $\pi$:

$$Q^\pi(s, a) = E_\pi[G_t | S_t = s, A_t = a]$$

The Q-function is more detailed - it tells us how good each action is in each state.

In [ ]:
# Compute state-value function for a policy (using MRP solution)
def compute_V_pi(T, R_sa, policy, gamma):
    """Compute V^π using direct solution of Bellman equation."""
    P_mrp, R_mrp = mdp_to_mrp(T, R_sa, policy)
    n_states = len(R_mrp)
    I = np.eye(n_states)
    V = np.linalg.solve(I - gamma * P_mrp, R_mrp)
    return V

# Compute Q from V
def compute_Q_from_V(T, R_sa, V, gamma):
    """Compute Q^π from V^π."""
    n_states, n_actions = R_sa.shape
    Q = np.zeros((n_states, n_actions))
    
    for s in range(n_states):
        for a in range(n_actions):
            # Q(s,a) = R(s,a) + γ * Σ P(s'|s,a) * V(s')
            Q[s, a] = R_sa[s, a] + gamma * np.sum(T[s, a] * V)
    
    return Q

# Compute values for random policy
gamma = 0.99
V_random = compute_V_pi(T, R, pi_random, gamma)
Q_random = compute_Q_from_V(T, R, V_random, gamma)

print(f"Value Functions for Random Policy (γ = {gamma})")
print("=" * 50)
print("\nState-Value Function V^π(s):")
for s in range(16):
    print(f"  V({s:2d}) = {V_random[s]:.4f}")

In [ ]:
# Visualize state values as a heatmap
def plot_value_function(V, env, title="State Value Function"):
    """Plot V as a heatmap on the FrozenLake grid."""
    desc = env.unwrapped.desc.astype(str)
    nrow, ncol = desc.shape
    
    # Reshape V to grid
    V_grid = V.reshape(nrow, ncol)
    
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Create heatmap
    im = ax.imshow(V_grid, cmap='RdYlGn', vmin=0, vmax=max(V.max(), 0.01))
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Value V(s)', rotation=270, labelpad=20)
    
    # Add text annotations
    for i in range(nrow):
        for j in range(ncol):
            cell = desc[i, j]
            state = i * ncol + j
            
            # Cell type
            ax.text(j, i - 0.25, cell, ha='center', va='center', 
                   fontsize=16, fontweight='bold')
            
            # Value
            ax.text(j, i + 0.25, f'{V[state]:.3f}', ha='center', va='center',
                   fontsize=10)
    
    ax.set_xticks(range(ncol))
    ax.set_yticks(range(nrow))
    ax.set_xticklabels(range(ncol))
    ax.set_yticklabels(range(nrow))
    ax.set_title(title, fontsize=14)
    
    plt.tight_layout()
    return fig, ax

plot_value_function(V_random, env, title=f"State Values V^π for Random Policy (γ={gamma})")
plt.show()

In [ ]:
# Visualize Q-values
def plot_q_values(Q, env, title="Q-Values"):
    """Plot Q-values showing the value of each action in each state."""
    desc = env.unwrapped.desc.astype(str)
    nrow, ncol = desc.shape
    n_actions = Q.shape[1]
    action_names = ['←', '↓', '→', '↑']
    
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Draw grid
    for i in range(nrow):
        for j in range(ncol):
            state = i * ncol + j
            cell = desc[i, j]
            
            # Background color based on cell type
            colors = {'S': 'lightblue', 'F': 'white', 'H': 'lightcoral', 'G': 'lightgreen'}
            rect = plt.Rectangle((j, nrow-1-i), 1, 1, fill=True,
                                 facecolor=colors.get(cell, 'white'), edgecolor='black')
            ax.add_patch(rect)
            
            # Add cell label
            ax.text(j + 0.5, nrow - 1 - i + 0.5, cell, ha='center', va='center',
                   fontsize=14, fontweight='bold', alpha=0.3)
            
            # Add Q-values for each action (if not hole or goal)
            if cell not in ['H', 'G']:
                best_action = np.argmax(Q[state])
                positions = [(0.25, 0.5), (0.5, 0.25), (0.75, 0.5), (0.5, 0.75)]  # L, D, R, U
                for a in range(n_actions):
                    x, y = positions[a]
                    color = 'green' if a == best_action else 'black'
                    weight = 'bold' if a == best_action else 'normal'
                    ax.text(j + x, nrow - 1 - i + y, f'{Q[state, a]:.2f}',
                           ha='center', va='center', fontsize=7, color=color, fontweight=weight)
    
    ax.set_xlim(0, ncol)
    ax.set_ylim(0, nrow)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(f"{title}\n(Green = best action, positions: ←left, ↓bottom, →right, ↑top)", fontsize=12)
    
    plt.tight_layout()
    return fig, ax

plot_q_values(Q_random, env, title="Q-Values for Random Policy")
plt.show()

---
# 6. Bellman Expectation Equations

The Bellman equations express the recursive relationship between value functions.

## Bellman Expectation Equation for $V^\pi$

$$V^\pi(s) = \sum_{a} \pi(a|s) \left[ R_s^a + \gamma \sum_{s'} P_{ss'}^a V^\pi(s') \right]$$

Or equivalently:

$$V^\pi(s) = \sum_{a} \pi(a|s) Q^\pi(s, a)$$

## Bellman Expectation Equation for $Q^\pi$

$$Q^\pi(s, a) = R_s^a + \gamma \sum_{s'} P_{ss'}^a V^\pi(s')$$

Or equivalently:

$$Q^\pi(s, a) = R_s^a + \gamma \sum_{s'} P_{ss'}^a \sum_{a'} \pi(a'|s') Q^\pi(s', a')$$

## Relationship Between V and Q

- $V^\pi(s) = \sum_a \pi(a|s) Q^\pi(s,a)$ — V is the expected Q over actions
- $Q^\pi(s,a) = R_s^a + \gamma \sum_{s'} P_{ss'}^a V^\pi(s')$ — Q is immediate reward plus discounted future V

In [ ]:
# Verify Bellman Expectation Equation for a specific state
state = 6  # A non-terminal state in the middle

print(f"Verifying Bellman Expectation Equation at State {state}")
print("=" * 60)
print(f"\nUsing random policy with γ = {gamma}")

# Method 1: Directly computed V(s)
V_direct = V_random[state]
print(f"\nDirect computation: V^π({state}) = {V_direct:.6f}")

# Method 2: Using Bellman equation
# V(s) = Σ_a π(a|s) * Q(s,a)
V_bellman = 0
print(f"\nBellman equation: V^π(s) = Σ_a π(a|s) * Q^π(s,a)")
for a in range(n_actions):
    contribution = pi_random[state, a] * Q_random[state, a]
    V_bellman += contribution
    print(f"  π({action_names[a]}|{state}) × Q({state},{action_names[a]}) = {pi_random[state,a]:.2f} × {Q_random[state,a]:.4f} = {contribution:.6f}")

print(f"\nSum: V^π({state}) = {V_bellman:.6f}")
print(f"\n✓ Verified!" if abs(V_direct - V_bellman) < 1e-6 else "✗ Error!")

In [ ]:
# Verify Q = R + γ * Σ P * V
state = 6
action = 2  # RIGHT

print(f"\nVerifying Q = R + γΣP·V at State {state}, Action {action_names[action]}")
print("=" * 60)

# Direct Q value
Q_direct = Q_random[state, action]
print(f"\nDirect computation: Q^π({state},{action_names[action]}) = {Q_direct:.6f}")

# Using Bellman equation
R_sa = R[state, action]
expected_V = 0
print(f"\nBellman equation: Q(s,a) = R(s,a) + γ * Σ P(s'|s,a) * V(s')")
print(f"\nR({state},{action_names[action]}) = {R_sa:.4f}")
print(f"\nExpected future value:")
for next_s in range(n_states):
    if T[state, action, next_s] > 0:
        contribution = T[state, action, next_s] * V_random[next_s]
        expected_V += contribution
        print(f"  P({next_s}|{state},{action_names[action]}) × V({next_s}) = {T[state,action,next_s]:.2f} × {V_random[next_s]:.4f} = {contribution:.6f}")

Q_bellman = R_sa + gamma * expected_V
print(f"\nQ({state},{action_names[action]}) = {R_sa:.4f} + {gamma} × {expected_V:.6f} = {Q_bellman:.6f}")
print(f"\n✓ Verified!" if abs(Q_direct - Q_bellman) < 1e-6 else "✗ Error!")

---
# 7. Optimal Value Functions

The **optimal state-value function** $V^*(s)$ is the maximum value function over all policies:

$$V^*(s) = \max_\pi V^\pi(s)$$

The **optimal action-value function** $Q^*(s, a)$ is the maximum action-value function over all policies:

$$Q^*(s, a) = \max_\pi Q^\pi(s, a)$$

## Key Theorem

For any MDP:
1. There exists an optimal policy $\pi^*$ that is better than or equal to all other policies
2. All optimal policies achieve the same optimal value functions $V^*$ and $Q^*$
3. There always exists a **deterministic** optimal policy

## Finding the Optimal Policy from $Q^*$

If we know $Q^*(s, a)$, the optimal policy is simply:

$$\pi^*(a|s) = \begin{cases} 1 & \text{if } a = \arg\max_{a'} Q^*(s, a') \\ 0 & \text{otherwise} \end{cases}$$

Just pick the action with the highest Q-value!

## Bellman Optimality Equations

The optimal value functions satisfy the **Bellman Optimality Equations**:

### For $V^*$:

$$V^*(s) = \max_a \left[ R_s^a + \gamma \sum_{s'} P_{ss'}^a V^*(s') \right]$$

### For $Q^*$:

$$Q^*(s, a) = R_s^a + \gamma \sum_{s'} P_{ss'}^a \max_{a'} Q^*(s', a')$$

### Key Difference from Bellman Expectation Equations:
- Expectation equations: **sum** over actions weighted by policy
- Optimality equations: **max** over actions

The **max** makes these equations **non-linear**, so we can't solve them directly with matrix inversion!

In [ ]:
# Value Iteration to find V* (preview - we'll cover this in detail in notebook 03)
def value_iteration(T, R, gamma, theta=1e-8, max_iterations=1000):
    """Find optimal value function using Value Iteration."""
    n_states = T.shape[0]
    n_actions = T.shape[1]
    
    V = np.zeros(n_states)
    
    for iteration in range(max_iterations):
        V_new = np.zeros(n_states)
        
        for s in range(n_states):
            # V*(s) = max_a [R(s,a) + γ * Σ P(s'|s,a) * V*(s')]
            q_values = np.zeros(n_actions)
            for a in range(n_actions):
                q_values[a] = R[s, a] + gamma * np.sum(T[s, a] * V)
            V_new[s] = np.max(q_values)
        
        # Check convergence
        if np.max(np.abs(V_new - V)) < theta:
            print(f"Converged after {iteration + 1} iterations")
            break
        
        V = V_new
    
    return V

# Find optimal value function
V_star = value_iteration(T, R, gamma=0.99)

# Compute Q* from V*
Q_star = compute_Q_from_V(T, R, V_star, gamma=0.99)

# Extract optimal policy
pi_star = np.zeros((n_states, n_actions))
for s in range(n_states):
    best_action = np.argmax(Q_star[s])
    pi_star[s, best_action] = 1.0

print("\nOptimal Value Function V*:")
print(V_star.reshape(4, 4).round(4))

In [ ]:
# Visualize optimal values and policy
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot V*
desc = env.unwrapped.desc.astype(str)
nrow, ncol = desc.shape
V_grid = V_star.reshape(nrow, ncol)

im = axes[0].imshow(V_grid, cmap='RdYlGn', vmin=0, vmax=V_star.max())
plt.colorbar(im, ax=axes[0], label='V*(s)')

for i in range(nrow):
    for j in range(ncol):
        state = i * ncol + j
        axes[0].text(j, i, f'{desc[i,j]}\n{V_star[state]:.3f}', 
                    ha='center', va='center', fontsize=10)

axes[0].set_title('Optimal Value Function V*', fontsize=12)
axes[0].set_xticks(range(ncol))
axes[0].set_yticks(range(nrow))

# Plot optimal policy
action_arrows = ['←', '↓', '→', '↑']
colors = {'S': 'lightblue', 'F': 'white', 'H': 'lightcoral', 'G': 'lightgreen'}

for i in range(nrow):
    for j in range(ncol):
        state = i * ncol + j
        cell = desc[i, j]
        
        rect = plt.Rectangle((j, nrow-1-i), 1, 1, fill=True,
                             facecolor=colors.get(cell, 'white'), edgecolor='black')
        axes[1].add_patch(rect)
        
        # Add cell label and optimal action
        best_action = np.argmax(Q_star[state])
        if cell not in ['H', 'G']:
            axes[1].text(j + 0.5, nrow - 1 - i + 0.5, 
                        f'{cell}\n{action_arrows[best_action]}',
                        ha='center', va='center', fontsize=14, fontweight='bold')
        else:
            axes[1].text(j + 0.5, nrow - 1 - i + 0.5, cell,
                        ha='center', va='center', fontsize=14, fontweight='bold')

axes[1].set_xlim(0, ncol)
axes[1].set_ylim(0, nrow)
axes[1].set_aspect('equal')
axes[1].axis('off')
axes[1].set_title('Optimal Policy π*\n(arrows show best action)', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Compare random policy vs optimal policy performance
def evaluate_policy(env, policy, n_episodes=10000):
    """Evaluate a policy by running episodes."""
    total_rewards = []
    
    for _ in range(n_episodes):
        obs, _ = env.reset()
        episode_reward = 0
        done = False
        
        while not done:
            # Sample action from policy
            action = np.random.choice(env.action_space.n, p=policy[obs])
            obs, reward, terminated, truncated, _ = env.step(action)
            episode_reward += reward
            done = terminated or truncated
        
        total_rewards.append(episode_reward)
    
    return total_rewards

# Evaluate both policies
env = gym.make("FrozenLake-v1", is_slippery=True)

print("Policy Comparison")
print("=" * 50)

rewards_random = evaluate_policy(env, pi_random, n_episodes=10000)
rewards_optimal = evaluate_policy(env, pi_star, n_episodes=10000)

print(f"\nRandom Policy:")
print(f"  Success rate: {np.mean(rewards_random)*100:.2f}%")

print(f"\nOptimal Policy:")
print(f"  Success rate: {np.mean(rewards_optimal)*100:.2f}%")

print(f"\nImprovement: {(np.mean(rewards_optimal) - np.mean(rewards_random))*100:.2f} percentage points")

env.close()

In [ ]:
# Visualize the comparison
fig, ax = plt.subplots(figsize=(8, 5))

policies = ['Random Policy', 'Optimal Policy']
success_rates = [np.mean(rewards_random) * 100, np.mean(rewards_optimal) * 100]
colors = ['gray', 'green']

bars = ax.bar(policies, success_rates, color=colors, edgecolor='black')

ax.set_ylabel('Success Rate (%)')
ax.set_title('Random vs Optimal Policy on FrozenLake\n(10,000 episodes each)')
ax.set_ylim(0, 100)

# Add value labels
for bar, rate in zip(bars, success_rates):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
            f'{rate:.1f}%', ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

---
# Summary

In this notebook, we covered the mathematical foundations of reinforcement learning:

## 1. Markov Process
- Tuple $(S, P)$: states and transition probabilities
- Memoryless random process

## 2. Markov Reward Process (MRP)
- Tuple $(S, P, R, \gamma)$: adds rewards and discount factor
- **Return**: $G_t = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}$
- **Value function**: $v(s) = E[G_t | S_t = s]$
- **Bellman equation**: $v = R + \gamma P v$

## 3. Markov Decision Process (MDP)
- Tuple $(S, A, P, R, \gamma)$: adds actions
- **Policy**: $\pi(a|s)$ - how to choose actions
- **State-value**: $V^\pi(s)$ - expected return from state $s$
- **Action-value**: $Q^\pi(s,a)$ - expected return from state $s$ taking action $a$

## 4. Bellman Equations
- **Expectation equations**: Recursive definition of $V^\pi$ and $Q^\pi$
- **Optimality equations**: Define $V^*$ and $Q^*$ using **max** instead of sum

## 5. Optimal Policy
- $\pi^*(a|s) = 1$ if $a = \arg\max_{a'} Q^*(s, a')$
- Always exists and is deterministic

## Next Steps

In the next notebook (**03_dynamic_programming.ipynb**), we'll learn algorithms to solve MDPs:
- Policy Evaluation
- Policy Iteration
- Value Iteration

In [ ]:
print("Congratulations! You've completed Part 2 of the RL Tutorial!")
print("\nKey takeaways:")
print("- MDPs provide the mathematical framework for RL")
print("- Value functions tell us how good states/actions are")
print("- Bellman equations relate values recursively")
print("- Optimal values satisfy the Bellman optimality equations")
print("- The optimal policy selects the action with highest Q*")
print("\nNext: 03_dynamic_programming.ipynb")